In [1]:
import torch
import pandas as pd

from sklearn.model_selection import train_test_split
import spacy

from flair.models import TextClassifier
from flair.data import Sentence
from flair.embeddings import WordEmbeddings, FlairEmbeddings, DocumentRNNEmbeddings, \
                             ELMoEmbeddings, BertEmbeddings
from flair.data_fetcher import NLPTaskDataFetcher
from flair.models import TextClassifier
from flair.trainers import ModelTrainer

from sklearn.metrics import f1_score

torch.cuda.set_device('cuda:0')

In [2]:
data = pd.read_csv('data/Tweets.csv')

In [3]:
# the same strutification as for baseline
train_df, test_df = train_test_split(data, test_size=0.2, stratify=data[['airline_sentiment', 'airline']],
                               random_state=21)

In [4]:
train_df, val_df = train_test_split(train_df, test_size=0.15, stratify=train_df[['airline_sentiment', 'airline']],
                               random_state=21)

# Flair

https://github.com/zalandoresearch/flair

### Data preparation

Flair expects data in fasttext format

In [5]:
processed_train = pd.DataFrame()
processed_val = pd.DataFrame()
processed_test = pd.DataFrame()

In [6]:
processed_train['label'] = '__label__' + train_df['airline_sentiment']
processed_val['label'] = '__label__' + val_df['airline_sentiment']
processed_test['label'] = '__label__' + test_df['airline_sentiment']

In [7]:
nlp = spacy.load('en_core_web_sm')
def process_text(text):
    """
    replace http link with <link> token
    tokenize with spacy
    first token will be thrown as model can become biased because of disbalances
        in class destribution through companies
    """
    sp = nlp(text)
    tokens = [tok.text if 'http' not in tok.text else '<link>' for tok in sp]
    return ' '.join(tokens[1:])

In [13]:
processed_train['text'] = train_df['text'].apply(lambda x: process_text(x))
processed_val['text'] = val_df['text'].apply(lambda x: process_text(x))
processed_test['text'] = test_df['text'].apply(lambda x: process_text(x))

In [14]:
processed_train.head()

,label,text
8938,__label__negative,Got flight reschedule to flight form PIT to FL...
1935,__label__negative,", more lies ... <link>"
5517,__label__positive,I 'm on the 10:55 flight ! Everyone has been s...
2030,__label__negative,you have Cancelled Flightled my flight UA922 f...
13916,__label__negative,because you wo n't get our bags for us because...


In [25]:
processed_train.to_csv('data/fl_train.csv', sep='\t', index = False, header = False)
processed_val.to_csv('data/fl_val.csv', sep='\t', index = False, header = False)
processed_test.to_csv('data/fl_test.csv', sep='\t', index = False, header = False)

In [5]:
corpus = NLPTaskDataFetcher.load_classification_corpus('data/', test_file='fl_test.csv', 
                                                       dev_file='fl_val.csv', train_file='fl_train.csv')

2019-07-26 22:51:11,275 Reading data from data
2019-07-26 22:51:11,276 Train: data/fl_train.csv
2019-07-26 22:51:11,277 Dev: data/fl_val.csv
2019-07-26 22:51:11,277 Test: data/fl_test.csv


/home/egor/.local/lib/python3.6/site-packages/ipykernel_launcher.py:2: DeprecationWarning: Call to deprecated function (or staticmethod) load_classification_corpus. (Use 'flair.datasets' instead.) -- Deprecated since version 0.4.1.
  
/home/egor/.local/lib/python3.6/site-packages/flair/data_fetcher.py:447: DeprecationWarning: Call to deprecated function (or staticmethod) read_text_classification_file. (Use 'flair.datasets' instead.) -- Deprecated since version 0.4.1.
  max_tokens_per_doc=max_tokens_per_doc,
/home/egor/.local/lib/python3.6/site-packages/flair/data_fetcher.py:454: DeprecationWarning: Call to deprecated function (or staticmethod) read_text_classification_file. (Use 'flair.datasets' instead.) -- Deprecated since version 0.4.1.
  max_tokens_per_doc=max_tokens_per_doc,
/home/egor/.local/lib/python3.6/site-packages/flair/data_fetcher.py:463: DeprecationWarning: Call to deprecated function (or staticmethod) read_text_classification_file. (Use 'flair.datasets' instead.) -- Depr

In [151]:
word_embeddings = [WordEmbeddings('glove'), 
                   FlairEmbeddings('news-forward-fast'), 
                   FlairEmbeddings('news-backward-fast')]

In [152]:
document_embeddings = DocumentRNNEmbeddings(word_embeddings, hidden_size=512, 
                                             reproject_words=True, reproject_words_dimension=256)

In [153]:
classifier = TextClassifier(document_embeddings, 
                            label_dictionary=corpus.make_label_dictionary(), multi_label=False)

2019-07-26 18:29:28,246 {'positive', 'negative', 'neutral'}


In [154]:
trainer = ModelTrainer(classifier, corpus)

In [155]:
trainer.train('flair_res', max_epochs=10)

2019-07-26 18:29:31,063 ----------------------------------------------------------------------------------------------------
2019-07-26 18:29:31,066 Evaluation method: MICRO_F1_SCORE
2019-07-26 18:29:31,408 ----------------------------------------------------------------------------------------------------
2019-07-26 18:29:31,986 epoch 1 - iter 0/312 - loss 1.06414855
2019-07-26 18:29:35,716 epoch 1 - iter 31/312 - loss 0.88447172
2019-07-26 18:29:39,512 epoch 1 - iter 62/312 - loss 0.87452886
2019-07-26 18:29:43,269 epoch 1 - iter 93/312 - loss 0.87888454
2019-07-26 18:29:46,981 epoch 1 - iter 124/312 - loss 0.86957454
2019-07-26 18:29:50,708 epoch 1 - iter 155/312 - loss 0.85930194
2019-07-26 18:29:54,432 epoch 1 - iter 186/312 - loss 0.85537625
2019-07-26 18:29:59,383 epoch 1 - iter 217/312 - loss 0.84801682
2019-07-26 18:30:03,191 epoch 1 - iter 248/312 - loss 0.84300612
2019-07-26 18:30:06,904 epoch 1 - iter 279/312 - loss 0.83592021
2019-07-26 18:30:10,480 epoch 1 - iter 310/312 

2019-07-26 18:36:05,314 EPOCH 7 done: loss 0.6072 - lr 0.1000 - bad epochs 0
2019-07-26 18:36:11,874 DEV : loss 0.7949720621109009 - score 0.6397
2019-07-26 18:36:22,508 TEST : loss 0.7792453765869141 - score 0.639
2019-07-26 18:36:22,509 ----------------------------------------------------------------------------------------------------
2019-07-26 18:36:23,113 epoch 8 - iter 0/312 - loss 0.81578618
2019-07-26 18:36:28,560 epoch 8 - iter 31/312 - loss 0.59795221
2019-07-26 18:36:32,059 epoch 8 - iter 62/312 - loss 0.60628806
2019-07-26 18:36:35,703 epoch 8 - iter 93/312 - loss 0.59220185
2019-07-26 18:36:39,386 epoch 8 - iter 124/312 - loss 0.60805073
2019-07-26 18:36:42,861 epoch 8 - iter 155/312 - loss 0.60349353
2019-07-26 18:36:46,506 epoch 8 - iter 186/312 - loss 0.59922331
2019-07-26 18:36:51,664 epoch 8 - iter 217/312 - loss 0.60319823
2019-07-26 18:36:55,292 epoch 8 - iter 248/312 - loss 0.59996523
2019-07-26 18:36:58,885 epoch 8 - iter 279/312 - loss 0.59925836
2019-07-26 18:3

{'test_score': 0.7626,
 'dev_score_history': [0.6574,
  0.6056,
  0.6887,
  0.2425,
  0.4246,
  0.7405,
  0.6397,
  0.7268,
  0.724,
  0.7507],
 'train_loss_history': [0.8325606673382796,
  0.7656564963742708,
  0.7068537591168513,
  0.6805603679938194,
  0.6541746378135986,
  0.6404987799529082,
  0.6071634303109769,
  0.5986123392239022,
  0.5701950810467585,
  0.5519521091228876],
 'dev_loss_history': [tensor(0.8431, device='cuda:0'),
  tensor(0.8721, device='cuda:0'),
  tensor(0.8785, device='cuda:0'),
  tensor(2.0347, device='cuda:0'),
  tensor(1.0273, device='cuda:0'),
  tensor(0.6433, device='cuda:0'),
  tensor(0.7950, device='cuda:0'),
  tensor(0.7900, device='cuda:0'),
  tensor(0.7963, device='cuda:0'),
  tensor(0.6106, device='cuda:0')]}

In [156]:
classifier = TextClassifier.load('flair_res/best-model.pt')

2019-07-26 18:39:41,820 loading file flair_res/best-model.pt


In [157]:
test_sents = [Sentence(s) for s in test_df['text'].to_list()]

In [158]:
%%time
classifier.predict(test_sents);

CPU times: user 8.09 s, sys: 1.26 s, total: 9.34 s
Wall time: 9.34 s


[Sentence: "@AmericanAir Hi guys checking in US/AA 639 JFK PHX and renewed Admirals Club today." - 14 Tokens,
 Sentence: "@JetBlue yes Party is Over 😂" - 6 Tokens,
 Sentence: "@SouthwestAir Thanks a ton!" - 4 Tokens,
 Sentence: "@AmericanAir I left something on my flight from ORD to SFO earlier today and believe it's on the plane back at ORD now. Please help." - 25 Tokens,
 Sentence: "@USAirways thank you" - 3 Tokens,
 Sentence: "@united So far so good. Just stepped down in Denver. Next Stop Portland!" - 13 Tokens,
 Sentence: "@united if the car seat is lost @united should just reimburse me for a new one, this is not a pair of shoes, it's a necessity for my child" - 29 Tokens,
 Sentence: "@united I would have made it if you hadn't already booked me on another flight and Cancelled Flighted my original reservation." - 21 Tokens,
 Sentence: "@JetBlue you're killing me now. :-) You got me! #smitten #trueblue" - 11 Tokens,
 Sentence: "@USAirways ok thanks" - 3 Tokens,
 Sentence: "@AmericanA

In [159]:
label_dic = corpus.make_label_dictionary().item2idx

2019-07-26 18:39:52,707 {'positive', 'negative', 'neutral'}


In [160]:
pred_test_labels = [p.labels[0].value for p in test_sents]
test_labels = test_df['airline_sentiment']

# labels2numbers
pred_test_labels = list(map(lambda x: label_dic[x.encode("utf-8")], pred_test_labels))
test_labels = list(map(lambda x: label_dic[x.encode("utf-8")], test_labels))

The result is worse than vanila baseline. In case of macro f1 it really bad

In [161]:
f1_score(pred_test_labels, test_labels,  average='macro'), f1_score(pred_test_labels, test_labels,  average='micro')

(0.6242843987371614, 0.7486338797814208)

### More powerful embeddings

In [17]:
better_word_embeddings = [WordEmbeddings('glove'),
                          ELMoEmbeddings()]

In [18]:
document_embeddings = DocumentRNNEmbeddings(better_word_embeddings, hidden_size=512, 
                                             reproject_words=True, reproject_words_dimension=256)

In [19]:
classifier = TextClassifier(document_embeddings, 
                            label_dictionary=corpus.make_label_dictionary(), multi_label=False)

2019-07-26 21:51:46,462 {'neutral', 'positive', 'negative'}


In [20]:
trainer = ModelTrainer(classifier, corpus)

In [21]:
trainer.train('flair_res', max_epochs=10, save_final_model=False, mini_batch_size=64)

2019-07-26 21:51:46,472 ----------------------------------------------------------------------------------------------------
2019-07-26 21:51:46,473 Evaluation method: MICRO_F1_SCORE
2019-07-26 21:51:46,714 ----------------------------------------------------------------------------------------------------
2019-07-26 21:51:47,333 epoch 1 - iter 0/156 - loss 1.16175115
2019-07-26 21:51:50,870 epoch 1 - iter 15/156 - loss 1.12883091
2019-07-26 21:51:54,860 epoch 1 - iter 30/156 - loss 0.99693760
2019-07-26 21:51:58,396 epoch 1 - iter 45/156 - loss 0.95832121
2019-07-26 21:52:01,958 epoch 1 - iter 60/156 - loss 0.90762670
2019-07-26 21:52:06,016 epoch 1 - iter 75/156 - loss 0.87605761
2019-07-26 21:52:09,555 epoch 1 - iter 90/156 - loss 0.86886064
2019-07-26 21:52:13,208 epoch 1 - iter 105/156 - loss 0.84597047
2019-07-26 21:52:17,206 epoch 1 - iter 120/156 - loss 0.83121773
2019-07-26 21:52:20,677 epoch 1 - iter 135/156 - loss 0.82398792
2019-07-26 21:52:24,154 epoch 1 - iter 150/156 - l

2019-07-26 21:58:32,654 DEV : loss 0.5232689380645752 - score 0.7917
2019-07-26 21:58:43,890 TEST : loss 0.508441686630249 - score 0.7995
2019-07-26 21:58:46,656 ----------------------------------------------------------------------------------------------------
2019-07-26 21:58:47,372 epoch 8 - iter 0/156 - loss 0.31328574
2019-07-26 21:58:51,058 epoch 8 - iter 15/156 - loss 0.48205936
2019-07-26 21:58:54,636 epoch 8 - iter 30/156 - loss 0.50295200
2019-07-26 21:58:58,778 epoch 8 - iter 45/156 - loss 0.50476149
2019-07-26 21:59:02,252 epoch 8 - iter 60/156 - loss 0.49970588
2019-07-26 21:59:05,788 epoch 8 - iter 75/156 - loss 0.50130691
2019-07-26 21:59:09,779 epoch 8 - iter 90/156 - loss 0.49507476
2019-07-26 21:59:13,278 epoch 8 - iter 105/156 - loss 0.49279749
2019-07-26 21:59:16,757 epoch 8 - iter 120/156 - loss 0.49277121
2019-07-26 21:59:20,863 epoch 8 - iter 135/156 - loss 0.48949766
2019-07-26 21:59:24,396 epoch 8 - iter 150/156 - loss 0.48487478
2019-07-26 21:59:25,646 ------

{'test_score': 0.8081,
 'dev_score_history': [0.7245,
  0.7399,
  0.7701,
  0.7661,
  0.7513,
  0.7906,
  0.7917,
  0.7877,
  0.8036,
  0.8139],
 'train_loss_history': [0.8069264976642071,
  0.6456773819831702,
  0.6061508160753127,
  0.5743608289422133,
  0.5586135179186479,
  0.5359629631424562,
  0.5098659012180108,
  0.48335306585217136,
  0.47182420373727113,
  0.4453623888011162],
 'dev_loss_history': [tensor(0.6646, device='cuda:0'),
  tensor(0.6333, device='cuda:0'),
  tensor(0.5911, device='cuda:0'),
  tensor(0.6058, device='cuda:0'),
  tensor(0.6146, device='cuda:0'),
  tensor(0.5423, device='cuda:0'),
  tensor(0.5233, device='cuda:0'),
  tensor(0.5588, device='cuda:0'),
  tensor(0.5090, device='cuda:0'),
  tensor(0.5012, device='cuda:0')]}

In [6]:
classifier = TextClassifier.load('flair_res/best-model.pt')

2019-07-26 22:11:37,608 loading file flair_res/best-model.pt


In [7]:
test_sents = [Sentence(s) for s in test_df['text'].to_list()]

In [8]:
%%time
classifier.predict(test_sents);

CPU times: user 9.91 s, sys: 1.21 s, total: 11.1 s
Wall time: 11.1 s


[Sentence: "@AmericanAir Hi guys checking in US/AA 639 JFK PHX and renewed Admirals Club today." - 14 Tokens,
 Sentence: "@JetBlue yes Party is Over 😂" - 6 Tokens,
 Sentence: "@SouthwestAir Thanks a ton!" - 4 Tokens,
 Sentence: "@AmericanAir I left something on my flight from ORD to SFO earlier today and believe it's on the plane back at ORD now. Please help." - 25 Tokens,
 Sentence: "@USAirways thank you" - 3 Tokens,
 Sentence: "@united So far so good. Just stepped down in Denver. Next Stop Portland!" - 13 Tokens,
 Sentence: "@united if the car seat is lost @united should just reimburse me for a new one, this is not a pair of shoes, it's a necessity for my child" - 29 Tokens,
 Sentence: "@united I would have made it if you hadn't already booked me on another flight and Cancelled Flighted my original reservation." - 21 Tokens,
 Sentence: "@JetBlue you're killing me now. :-) You got me! #smitten #trueblue" - 11 Tokens,
 Sentence: "@USAirways ok thanks" - 3 Tokens,
 Sentence: "@AmericanA

In [9]:
label_dic = corpus.make_label_dictionary().item2idx

2019-07-26 22:12:04,845 {'negative', 'neutral', 'positive'}


In [10]:
pred_test_labels = [p.labels[0].value for p in test_sents]
test_labels = test_df['airline_sentiment']

# labels2numbers
pred_test_labels = list(map(lambda x: label_dic[x.encode("utf-8")], pred_test_labels))
test_labels = list(map(lambda x: label_dic[x.encode("utf-8")], test_labels))

Almost as good as  baseline. Now I see that SVM over TF-IDF is a really good thing for text classification.

In [11]:
f1_score(pred_test_labels, test_labels,  average='macro'), f1_score(pred_test_labels, test_labels,  average='micro')

(0.7199755261057943, 0.798155737704918)

### Even more powerfull embeddings

Combine all previous embedding and train for more epochs. Don't use bert because its too slow to train

In [6]:
better_word_embeddings = [WordEmbeddings('glove'),
                          FlairEmbeddings('news-forward'), 
                          FlairEmbeddings('news-backward'),
                          ELMoEmbeddings()]

/home/egor/.local/lib/python3.6/site-packages/sklearn/utils/linear_assignment_.py:21: DeprecationWarning: The linear_assignment_ module is deprecated in 0.21 and will be removed from 0.23. Use scipy.optimize.linear_sum_assignment instead.
  DeprecationWarning)


In [7]:
document_embeddings = DocumentRNNEmbeddings(better_word_embeddings, hidden_size=512, 
                                             reproject_words=True, reproject_words_dimension=256)

In [8]:
classifier = TextClassifier(document_embeddings, 
                            label_dictionary=corpus.make_label_dictionary(), multi_label=False)

2019-07-26 22:51:36,157 {'positive', 'negative', 'neutral'}


In [9]:
trainer = ModelTrainer(classifier, corpus)

In [10]:
trainer.train('flair_res_glove+news+elmo', max_epochs=15, save_final_model=False, mini_batch_size=64)

2019-07-26 22:52:25,339 ----------------------------------------------------------------------------------------------------
2019-07-26 22:52:25,339 Evaluation method: MICRO_F1_SCORE
2019-07-26 22:52:25,528 ----------------------------------------------------------------------------------------------------
2019-07-26 22:52:26,337 epoch 1 - iter 0/156 - loss 1.14448154
2019-07-26 22:52:34,174 epoch 1 - iter 15/156 - loss 1.11986494
2019-07-26 22:52:42,275 epoch 1 - iter 30/156 - loss 1.02042280
2019-07-26 22:52:50,089 epoch 1 - iter 45/156 - loss 0.95181172
2019-07-26 22:52:58,289 epoch 1 - iter 60/156 - loss 0.92574518
2019-07-26 22:53:05,756 epoch 1 - iter 75/156 - loss 0.90207891
2019-07-26 22:53:13,309 epoch 1 - iter 90/156 - loss 0.87923053
2019-07-26 22:53:21,361 epoch 1 - iter 105/156 - loss 0.85876696
2019-07-26 22:53:28,979 epoch 1 - iter 120/156 - loss 0.84499512
2019-07-26 22:53:37,235 epoch 1 - iter 135/156 - loss 0.82662319
2019-07-26 22:53:44,782 epoch 1 - iter 150/156 - l

2019-07-26 23:06:12,012 DEV : loss 0.5330607891082764 - score 0.7843
2019-07-26 23:06:35,967 TEST : loss 0.5186353325843811 - score 0.7958
2019-07-26 23:06:39,707 ----------------------------------------------------------------------------------------------------
2019-07-26 23:06:40,629 epoch 8 - iter 0/156 - loss 0.46281457
2019-07-26 23:06:48,403 epoch 8 - iter 15/156 - loss 0.47253022
2019-07-26 23:06:56,617 epoch 8 - iter 30/156 - loss 0.49907163
2019-07-26 23:07:04,103 epoch 8 - iter 45/156 - loss 0.49126696
2019-07-26 23:07:12,104 epoch 8 - iter 60/156 - loss 0.49392137
2019-07-26 23:07:19,770 epoch 8 - iter 75/156 - loss 0.48748342
2019-07-26 23:07:27,739 epoch 8 - iter 90/156 - loss 0.48822416
2019-07-26 23:07:35,308 epoch 8 - iter 105/156 - loss 0.48767232
2019-07-26 23:07:43,185 epoch 8 - iter 120/156 - loss 0.48244604
2019-07-26 23:07:50,577 epoch 8 - iter 135/156 - loss 0.47912649
2019-07-26 23:07:58,558 epoch 8 - iter 150/156 - loss 0.49130548
2019-07-26 23:08:01,073 -----

2019-07-26 23:20:08,721 EPOCH 14 done: loss 0.3936 - lr 0.1000 - bad epochs 3
2019-07-26 23:20:23,605 DEV : loss 0.5272204875946045 - score 0.8031
2019-07-26 23:20:48,068 TEST : loss 0.5040757060050964 - score 0.8098
2019-07-26 23:20:51,424 ----------------------------------------------------------------------------------------------------
2019-07-26 23:20:52,402 epoch 15 - iter 0/156 - loss 0.36014715
2019-07-26 23:21:00,479 epoch 15 - iter 15/156 - loss 0.34668834
2019-07-26 23:21:08,581 epoch 15 - iter 30/156 - loss 0.37158347
2019-07-26 23:21:16,266 epoch 15 - iter 45/156 - loss 0.36599717
2019-07-26 23:21:24,225 epoch 15 - iter 60/156 - loss 0.36075935
2019-07-26 23:21:31,987 epoch 15 - iter 75/156 - loss 0.36926989
2019-07-26 23:21:39,885 epoch 15 - iter 90/156 - loss 0.36558069
2019-07-26 23:21:47,567 epoch 15 - iter 105/156 - loss 0.36228029
2019-07-26 23:21:55,480 epoch 15 - iter 120/156 - loss 0.36683678
2019-07-26 23:22:03,013 epoch 15 - iter 135/156 - loss 0.37650092
2019-0

{'test_score': 0.8173,
 'dev_score_history': [0.7211,
  0.6898,
  0.6949,
  0.749,
  0.667,
  0.7348,
  0.7843,
  0.7638,
  0.7684,
  0.7906,
  0.6636,
  0.7661,
  0.7809,
  0.8031,
  0.8071],
 'train_loss_history': [0.8139249827617254,
  0.6435403485710804,
  0.5981274955929854,
  0.5721451332553839,
  0.5549242282525088,
  0.5357326656962053,
  0.5218328459140582,
  0.48986387978761625,
  0.47603633598639417,
  0.4627191405265759,
  0.4373194359433957,
  0.4246067287257084,
  0.4062405588726203,
  0.39357075926202995,
  0.37413827711955094],
 'dev_loss_history': [tensor(0.6534, device='cuda:0'),
  tensor(0.8320, device='cuda:0'),
  tensor(0.7111, device='cuda:0'),
  tensor(0.6390, device='cuda:0'),
  tensor(0.8022, device='cuda:0'),
  tensor(0.6539, device='cuda:0'),
  tensor(0.5331, device='cuda:0'),
  tensor(0.5869, device='cuda:0'),
  tensor(0.7116, device='cuda:0'),
  tensor(0.5648, device='cuda:0'),
  tensor(0.8479, device='cuda:0'),
  tensor(0.5733, device='cuda:0'),
  tensor(0

In [11]:
test_sents = [Sentence(s) for s in test_df['text'].to_list()]

In [12]:
%%time
classifier.predict(test_sents);

CPU times: user 22.4 s, sys: 4.27 s, total: 26.6 s
Wall time: 26.6 s


[Sentence: "@AmericanAir Hi guys checking in US/AA 639 JFK PHX and renewed Admirals Club today." - 14 Tokens,
 Sentence: "@JetBlue yes Party is Over 😂" - 6 Tokens,
 Sentence: "@SouthwestAir Thanks a ton!" - 4 Tokens,
 Sentence: "@AmericanAir I left something on my flight from ORD to SFO earlier today and believe it's on the plane back at ORD now. Please help." - 25 Tokens,
 Sentence: "@USAirways thank you" - 3 Tokens,
 Sentence: "@united So far so good. Just stepped down in Denver. Next Stop Portland!" - 13 Tokens,
 Sentence: "@united if the car seat is lost @united should just reimburse me for a new one, this is not a pair of shoes, it's a necessity for my child" - 29 Tokens,
 Sentence: "@united I would have made it if you hadn't already booked me on another flight and Cancelled Flighted my original reservation." - 21 Tokens,
 Sentence: "@JetBlue you're killing me now. :-) You got me! #smitten #trueblue" - 11 Tokens,
 Sentence: "@USAirways ok thanks" - 3 Tokens,
 Sentence: "@AmericanA

In [13]:
label_dic = corpus.make_label_dictionary().item2idx

2019-07-26 23:26:21,452 {'positive', 'negative', 'neutral'}


In [14]:
pred_test_labels = [p.labels[0].value for p in test_sents]
test_labels = test_df['airline_sentiment']

# labels2numbers
pred_test_labels = list(map(lambda x: label_dic[x.encode("utf-8")], pred_test_labels))
test_labels = list(map(lambda x: label_dic[x.encode("utf-8")], test_labels))

Again something similar to baseline score. Only macro f1 is higher now. But inference for the same data now need twice more time in comparison to using only glove and elmo embeddings.

In [15]:
f1_score(pred_test_labels, test_labels,  average='macro'), f1_score(pred_test_labels, test_labels,  average='micro')

(0.7373554699428003, 0.79474043715847)

So I think method used in flair is not enogh for this task.